# A very simple example

Here I will implement the very simple example from section Subsection 3.1 of my Ideas for the Target paper. See the full details there. 

The general problem can be stated here.
$$
\begin{gather}
    \nonumber \text{Given that:} \\
    \nonumber g_0 m_0 + g_1 m_1 = d \\
    \nonumber \text{Find} \\
    \nonumber p = t_0 m_0 + t_1 m_1 \\
    \nonumber \text{Where:} \\
    \nonumber m_0, m_1 \in \mathbb{R} - \text{Model} \\
    \nonumber g_0, g_1 \in \mathbb{R} - \text{"Sensitivity Kernel"}\\
    \nonumber t_0, t_1 \in \mathbb{R} - \text{"Property"} \\
    \nonumber d \in \mathbb{R} - \text{Data}
\end{gather}
$$

For this practical example we need to give some actual values to all the variables assumed to be knwon:
1. $g_0=1, g_1=2$ 
2. $t_0=1, t_1=1.99$

Let's assume that the true model is $\bar{m} = [10,15]^T$, therefore

3. $d=40$
4. $\bar{p} = 39.85$ - This is the true property, what we would like to find.

Without any norm bound it is obvious that our property p can take any value. So we have to introduce a norm bound $M$. The norm of the true model is $\lVert \bar{m} \rVert_{\mathcal{M}} = \sqrt{325} \approx 18.03$, so to emulate the fact that in reality we cannot know the true norm, we will take the norm bound to be 5 times bigger than the true norm: 

5. $M=5\sqrt{325} \approx 90.14$

First step is to set up the spaces and transformations:

In [1]:
import sys
sys.path.append('/home/adrian/PhD/BGSOLA/SOLA_DLI/core')
from core.main_classes.spaces import RN
from core.main_classes.mappings import FiniteLinearMapping

import numpy as np

In [2]:
# model space
M = RN(dimension=2)
# Data space
D = RN(dimension=1)
# Property space
P = RN(dimension=1)

# Data mapping (a row vector)
G = FiniteLinearMapping(domain=M, 
                        codomain=D,
                        matrix=np.array([[1, 2]]))
# Property mapping (also a row vector)
T = FiniteLinearMapping(domain=M, 
                        codomain=P,
                        matrix=np.array([[1, 1.99]]))
# Data
data = np.array(40)

Now we find the adjoint mappings

In [3]:
G_adjoint = G.adjoint()
T_adjoint = T.adjoint()

The general solution to the data constraint is 
$$
\begin{gather}
    \nonumber \bar{m} \in G^T(GG^T)^{-1}d + \ker(G)\\
    \nonumber \bar{m} \in [g_0, g_1]^T \frac{d}{g_0^2 + g_1^2} + \ker(G)
\end{gather}
$$
where $ G^T(GG^T)^{-1}d$ is the least norm solution. We can find this solution with our mappings.

In [4]:
# Find pseudo-inverse
G_pseudo_inverse = G_adjoint * (G * G_adjoint).invert()
print('Pseudo-inverse mapping: \n', G_pseudo_inverse.matrix)

# Find least norm model solution
m_tilde = G_pseudo_inverse.map(data)
print('Least norm model solution: \n', m_tilde)

Pseudo-inverse mapping: 
 [[0.2]
 [0.4]]
Least norm model solution: 
 [[ 8.]
 [16.]]


As we can see, the least norm model solution is not too far from the true model solution $[10,15]^T$. But we are not here for the model solution, we want to find the property bounds on $p$. For this, we need the matrix $\mathcal{H} = T - RG$. We know $T$ and $G$, so we need to find $R$. But $R=TG^*(GG^*)^{-1}$, so it si simply $T$ composed with the G pseudo-inverse mapping.

In [5]:
R = T * G_pseudo_inverse
print(R.matrix)

[[0.996]]


So we can finally get 

In [6]:
H = T - R*G

print(H.matrix)

[[ 0.004 -0.002]]
